In [52]:
import cv2

if hasattr(cv2, "aruco"):
    print("Aruco is available.")


Aruco is available.


In [53]:
# %% ───────────────  QR-DRIVEN ArUco tracker + JSON export  ───────────────
import cv2
import json
import numpy as np
import logging

# ─── I/O ──────────────────────────────────────────────────────────────────
VIDEO_IN = "aruco.mp4"
VIDEO_OUT = "aruco_green_overlay.mp4"  # quick visual check (optional)
JSON_OUT = "aruco_screen_tracks.json"  # <-- this is what you will load later
ALPHA = 0.30  # overlay transparency

# ─── logging ──────────────────────────────────────────────────────────────
logging.basicConfig(
    filename="aruco_debug.log", filemode="w", level=logging.DEBUG, format="%(message)s"
)
log = logging.getLogger("aruco")

# ═════════ 0.  pull metadata QR (fail if none) ════════════════════════════
cap = cv2.VideoCapture(VIDEO_IN)
qr = cv2.QRCodeDetector()

qr_meta = None
while True:
    ok, frm = cap.read()
    if not ok:
        break
    payload, *_ = qr.detectAndDecode(frm)
    if payload:
        qr_meta = json.loads(payload)
        break

if qr_meta is None:
    raise RuntimeError("No metadata QR code found in the video")

for k in (
    "dictionary",
    "margin_px",
    "marker_size_px",
    "corner_markers",
    "screen_aspect",
):
    if k not in qr_meta:
        raise RuntimeError(f"QR metadata missing key: {k}")

# ─── constants from QR ────────────────────────────────────────────────────
dict_name = qr_meta["dictionary"]
dictionary = cv2.aruco.getPredefinedDictionary(getattr(cv2.aruco, dict_name))
margin_px = int(qr_meta["margin_px"])

corner_map = {  # marker-id → marker-corner-index (0-3)
    qr_meta["corner_markers"]["TL"]["id"]: qr_meta["corner_markers"]["TL"][
        "corner_index"
    ],
    qr_meta["corner_markers"]["TR"]["id"]: qr_meta["corner_markers"]["TR"][
        "corner_index"
    ],
    qr_meta["corner_markers"]["BR"]["id"]: qr_meta["corner_markers"]["BR"][
        "corner_index"
    ],
    qr_meta["corner_markers"]["BL"]["id"]: qr_meta["corner_markers"]["BL"][
        "corner_index"
    ],
}

# neighbours inside one marker (OpenCV order 0-1-2-3)
NEIGH_H = {0: 1, 1: 0, 2: 3, 3: 2}
NEIGH_V = {0: 3, 1: 2, 2: 1, 3: 0}


def screen_corner_from_marker(c4, inner):
    """Shift the inner marker-corner outwards by `margin_px` along marker edges."""
    c, h, v = c4[inner], c4[NEIGH_H[inner]], c4[NEIGH_V[inner]]
    u_h = (c - h) / np.linalg.norm(c - h)
    u_v = (c - v) / np.linalg.norm(c - v)
    diag = (u_h + u_v) / np.linalg.norm(u_h + u_v)  # unit vector toward screen corner
    return (c + margin_px * diag).astype(np.float32)


# ═════════ 1.  iterate video, collect quads, draw green overlay ═══════════
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # rewind

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_vid = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w, h))
detector = cv2.aruco.ArucoDetector(dictionary, cv2.aruco.DetectorParameters())

frames_json = []  # list of dicts (one per frame)
f_idx = 0
while True:
    ok, frame = cap.read()
    if not ok:
        break
    quad = np.zeros((4, 2), np.float32)
    detected = [False] * 4

    corners, ids, _ = detector.detectMarkers(frame)
    if ids is not None:
        for idx, mid in enumerate(ids.flatten()):
            if mid in corner_map:
                inner = corner_map[mid]
                quad[inner] = screen_corner_from_marker(corners[idx][0], inner)
                detected[inner] = True

    frame_record = {"valid": bool(all(detected))}
    if frame_record["valid"]:
        frame_record["corners"] = quad.tolist()  # [[xTL,yTL], …, [xBL,yBL]]

        # quick visual check (optional)
        mask = np.zeros_like(frame)
        cv2.fillPoly(mask, [quad.astype(np.int32)], (0, 255, 0))
        frame = cv2.addWeighted(frame, 1 - ALPHA, mask, ALPHA, 0)
        cv2.polylines(frame, [quad.astype(np.int32)], True, (0, 255, 0), 2)

    frames_json.append(frame_record)
    out_vid.write(frame)
    f_idx += 1

cap.release()
out_vid.release()
print(f"Overlay video written → {VIDEO_OUT}")

# ═════════ 2.  dump JSON with everything needed for later replacement ════
export = {
    "dictionary": dict_name,
    "margin_px": qr_meta["margin_px"],
    "marker_size_px": qr_meta["marker_size_px"],
    "screen_aspect": qr_meta["screen_aspect"],
    "frame_width": w,
    "frame_height": h,
    "corner_markers": qr_meta["corner_markers"],
    "frames": frames_json,  # one entry per video frame
}

with open(JSON_OUT, "w", encoding="utf-8") as f:
    json.dump(export, f, separators=(",", ":"))  # compact, ~30-40 MB per minute @30 fps
print(f"📄  JSON track file saved → {JSON_OUT}")


Overlay video written → aruco_green_overlay.mp4
📄  JSON track file saved → aruco_screen_tracks.json
